In [1]:
!nvidia-smi

Sat Jan 15 13:34:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://www.dropbox.com/s/bg4p13u0p6h6u47/Dataset_3_channel.zip?dl=0

--2022-01-15 13:34:14--  https://www.dropbox.com/s/bg4p13u0p6h6u47/Dataset_3_channel.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/bg4p13u0p6h6u47/Dataset_3_channel.zip [following]
--2022-01-15 13:34:14--  https://www.dropbox.com/s/raw/bg4p13u0p6h6u47/Dataset_3_channel.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc798e3aa8992d109cf078088f53.dl.dropboxusercontent.com/cd/0/inline/Bd0_TFVTJHDOF69O0fGHnakvoTIsl932f3kkt1x952ulbAmRfHrsDKBlBtzkojzwczN1sGJmdPELQ8bY2RYOYJ_O6bY71xRceRJ_uqhJXuRbDZRsJSwLb1cv9Ds5jZvZ26I_JMuJmzePZARNUgEt2RCZ/file# [following]
--2022-01-15 13:34:14--  https://uc798e3aa8992d109cf078088f53.dl.dropboxusercontent.com/cd/0/inline/Bd0_TFVTJHDOF69O0fGHnakvoTIsl932f3kkt1x952ulbAmRfHrsDKBlBt

In [3]:
!unzip -q "/content/Dataset_3_channel.zip?dl=0"

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
#from livelossplot import PlotLossesKerasTF

In [5]:
img_height = 300
img_width = 300
batch_size = 32

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1) # set validation split

In [7]:
from pathlib import Path
# Creating the Pathlib PATH objects
train_data_dir= Path("/content/Dataset_3_channel/Train")

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data


validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

Found 1355 images belonging to 2 classes.
Found 150 images belonging to 2 classes.


In [9]:
# Building ResNet50 model
from tensorflow.keras.applications import ResNet50V2

backbone = ResNet50V2(
    input_shape=(300, 300, 3),
    include_top=False
)

model = tf.keras.Sequential([
    backbone,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

94683136/94668760 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 10, 10, 2048)      23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23,566,849
Trainable params: 23,521,409
Non-trainable params: 45,440
_________________________________________________________________


In [10]:
# Compiling your model by providing the Optimizer , Loss and Metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
    loss = 'binary_crossentropy',
    metrics=['accuracy' , tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')]
)

In [11]:
# Defining our callbacks 
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_weights_res3.h5",verbose=1,save_best_only=True,save_weights_only = True)
# if false then it will save for every iteration

early_stop = tf.keras.callbacks.EarlyStopping(patience=10)

In [12]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=60,
    callbacks=[checkpoint, early_stop],
    validation_data=validation_generator,
    validation_steps = validation_generator.samples // batch_size,
)

Epoch 1/60
42/42 [==============================] - ETA: 0s - loss: 0.4815 - accuracy: 0.7770 - precision: 0.7997 - recall: 0.7352
Epoch 00001: val_loss improved from inf to 43732.16406, saving model to best_weights_res3.h5
42/42 [==============================] - 44s 615ms/step - loss: 0.4815 - accuracy: 0.7770 - precision: 0.7997 - recall: 0.7352 - val_loss: 43732.1641 - val_accuracy: 0.5078 - val_precision: 0.5078 - val_recall: 1.0000
Epoch 2/60
42/42 [==============================] - ETA: 0s - loss: 0.4201 - accuracy: 0.8095 - precision: 0.8887 - recall: 0.7047
Epoch 00002: val_loss improved from 43732.16406 to 97.49271, saving model to best_weights_res3.h5
42/42 [==============================] - 23s 537ms/step - loss: 0.4201 - accuracy: 0.8095 - precision: 0.8887 - recall: 0.7047 - val_loss: 97.4927 - val_accuracy: 0.5469 - val_precision: 0.5243 - val_recall: 0.8571
Epoch 3/60
42/42 [==============================] - ETA: 0s - loss: 0.3053 - accuracy: 0.8668 - precision: 0.9243 

    42/42 [==============================] - 23s 549ms/step - loss: 0.1084 - accuracy: 0.9577 - precision: 0.9679 - 
    recall: 0.9476 - val_loss: 0.5081 - val_accuracy: 0.8750 - val_precision: 0.9000 - val_recall: 0.8438

In [3]:
prect = 0.9000
rect = 0.8438

In [4]:
F1t = 2*((prect*rect)/(prect+rect))
F1t

0.8709943800894597

In [ ]:
# Interpreting the Metrics 
fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate(['precision', 'recall', 'accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [13]:
# Saving the best Model

# Load the best weights
model.load_weights("best_weights_res3.h5")
# Save the whole model (weigths + architecture)

model.save("model_res3.h5")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [14]:
# Loading the whole model
loaded_model = tf.keras.models.load_model("model_res3.h5")

In [15]:
test_data_dir= Path("/content/Dataset_3_channel/Test")

In [16]:
test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0)

In [19]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='binary',
    subset='training')

Found 320 images belonging to 2 classes.


In [21]:
# Evaluating the loaded model
loss, acc, prec, rec = loaded_model.evaluate(test_generator)

print(" Testing Acc : " , acc)
print(" Testing Precision " , prec)
print(" Testing Recall " , rec)

320/320 [==============================] - 8s 18ms/step - loss: 0.5568 - accuracy: 0.8188 - precision: 0.7602 - recall: 0.9312
 Testing Acc :  0.8187500238418579
 Testing Precision  0.7602040767669678
 Testing Recall  0.9312499761581421


In [5]:
p = 0.7602
r = 0.9312
F1 = 2*((p*r)/(p+r))
F1

0.8370559772969138